# Day 18: Snailfish

In [1]:
example = """[[[0,[5,8]],[[1,7],[9,6]]],[[4,[1,2]],[[1,4],2]]]
[[[5,[2,8]],4],[5,[[9,9],0]]]
[6,[[[6,2],[5,6]],[[7,6],[4,7]]]]
[[[6,[0,7]],[0,9]],[4,[9,[9,0]]]]
[[[7,[6,4]],[3,[1,3]]],[[[5,5],1],9]]
[[6,[[7,3],[3,2]]],[[[3,8],[5,7]],4]]
[[[[5,4],[7,7]],8],[[8,3],8]]
[[9,3],[[9,9],[6,[4,9]]]]
[[2,[[7,7],7]],[[5,8],[[9,3],[0,2]]]]
[[[[5,2],5],[8,[3,7]]],[[5,[7,5]],[4,4]]]"""

In [2]:
def parse(input):
    """Parse snailfish numbers from semi-trusted input ;)"""
    return [eval(line) for line in input.splitlines()]

parse(example)

[[[[0, [5, 8]], [[1, 7], [9, 6]]], [[4, [1, 2]], [[1, 4], 2]]],
 [[[5, [2, 8]], 4], [5, [[9, 9], 0]]],
 [6, [[[6, 2], [5, 6]], [[7, 6], [4, 7]]]],
 [[[6, [0, 7]], [0, 9]], [4, [9, [9, 0]]]],
 [[[7, [6, 4]], [3, [1, 3]]], [[[5, 5], 1], 9]],
 [[6, [[7, 3], [3, 2]]], [[[3, 8], [5, 7]], 4]],
 [[[[5, 4], [7, 7]], 8], [[8, 3], 8]],
 [[9, 3], [[9, 9], [6, [4, 9]]]],
 [[2, [[7, 7], 7]], [[5, 8], [[9, 3], [0, 2]]]],
 [[[[5, 2], 5], [8, [3, 7]]], [[5, [7, 5]], [4, 4]]]]

In [3]:
def lookup(snailfish, address):
    """Return value of address in snailfish."""
    for index in address:
        snailfish = snailfish[index]
    return snailfish

lookup([[1, 2], [[3, 4], 5]], [1, 1])

5

In [4]:
LEFT = 0
RIGHT = 1

def walk(snailfish, address=None):
    """Depth first walk.
    
    Yields value and address of each ordinary number.
    """
    if address is None:
        address = []
    try:
        left, right = snailfish
        yield from walk(left, address=address + [LEFT])
        yield from walk(right, address=address + [RIGHT])
    except TypeError:
        yield snailfish, address

list(walk([[[[[9,8],1],2],3],4]))

[(9, [0, 0, 0, 0, 0]),
 (8, [0, 0, 0, 0, 1]),
 (1, [0, 0, 0, 1]),
 (2, [0, 0, 1]),
 (3, [0, 1]),
 (4, [1])]

Test exploding examples.

In [5]:
def explode(snailfish):
    """Explodes snailfish (in-place).
    
    Returns True if exploded."""
    # Track previously visited address so it can be incremented once exploded.
    prev_address = None
    # Note address of exploding pair (not value in pair).
    exploding_address = None
    for value, address in walk(snailfish):
        # If a pair has exploded, increment the next value by the right side's value.
        if exploding_address and exploding_address + [RIGHT] != address:
            lookup(snailfish, address[:-1])[address[-1]] += lookup(snailfish, exploding_address + [RIGHT])
            break
            
        # The address will exceed length 4 on left value of exploding pair.
        if len(address) > 4 and address[-1] == LEFT:
            exploding_address = address[:-1]
            if prev_address:
                lookup(snailfish, prev_address[:-1])[prev_address[-1]] += value
                
        prev_address = address

    # Replace exploding pair with 0.
    if exploding_address:
        lookup(snailfish, exploding_address[:-1])[exploding_address[-1]] = 0
        return True
    else:
        return False

snailfish = [[[[[9,8],1],2],3],4]
explode(snailfish)
assert snailfish == [[[[0,9],2],3],4]

snailfish = [7,[6,[5,[4,[3,2]]]]]  
explode(snailfish)
assert snailfish == [7,[6,[5,[7,0]]]]

snailfish = [[6,[5,[4,[3,2]]]],1]
explode(snailfish)
assert snailfish == [[6,[5,[7,0]]],3]

snailfish = [[3,[2,[1,[7,3]]]],[6,[5,[4,[3,2]]]]]
explode(snailfish)
assert snailfish == [[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]

snailfish = [[3,[2,[8,0]]],[9,[5,[4,[3,2]]]]]
explode(snailfish)
assert snailfish == [[3,[2,[8,0]]],[9,[5,[7,0]]]]

Test split examples.

In [6]:
def split(snailfish):
    """"""
    for value, address in walk(snailfish):
        if value >= 10:
            lookup(snailfish, address[:-1])[address[-1]] = [value // 2, value - value // 2]
            return True
    else:
        return False

snailfish = [10, 1] 
split(snailfish)
assert snailfish == [[5,5],1]

snailfish = [11, 1] 
split(snailfish)
assert snailfish == [[5,6],1]

snailfish = [12, 1] 
split(snailfish)
assert snailfish == [[6,6],1]

Test addition.

In [7]:
import copy

def add(left, right):
    """Returns the addition of two snailfish numbers."""
    snailfish = [copy.deepcopy(left), copy.deepcopy(right)]
    while explode(snailfish) or split(snailfish):
        pass
    return snailfish

assert add([1,2], [[3,4],5]) == [[1,2],[[3,4],5]]
assert add([[[[4,3],4],4],[7,[[8,4],9]]], [1,1]) == [[[[0,7],4],[[7,8],[6,0]]],[8,1]]

Test summation.

In [8]:
def sum(snailfishies):
    """Returns summation of snailfishies."""
    total = snailfishies[0]
    [total := add(total, snailfish) for snailfish in snailfishies[1:]]
    return total

assert sum(parse("""[1,1]
[2,2]
[3,3]
[4,4]""")) == [[[[1,1],[2,2]],[3,3]],[4,4]]

assert sum(parse("""[1,1]
[2,2]
[3,3]
[4,4]
[5,5]""")) == [[[[3,0],[5,3]],[4,4]],[5,5]]

assert sum(parse("""[1,1]
[2,2]
[3,3]
[4,4]
[5,5]
[6,6]""")) == [[[[5,0],[7,4]],[5,5]],[6,6]]

assert sum(parse("""[[[0,[4,5]],[0,0]],[[[4,5],[2,6]],[9,5]]]
[7,[[[3,7],[4,3]],[[6,3],[8,8]]]]
[[2,[[0,8],[3,4]]],[[[6,7],1],[7,[1,6]]]]
[[[[2,4],7],[6,[0,5]]],[[[6,8],[2,8]],[[2,1],[4,5]]]]
[7,[5,[[3,8],[1,4]]]]
[[2,[2,2]],[8,[8,1]]]
[2,9]
[1,[[[9,3],9],[[9,0],[0,7]]]]
[[[5,[7,4]],7],1]
[[[[4,2],2],6],[8,7]]""")) == [[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]

Test magnitude.

In [9]:
def magnitude(snailfish):
    """Returns magnitude of snailfish number."""
    try:
        left, right = snailfish
    except TypeError:
        return snailfish
    
    return 3 * magnitude(left) + 2 * magnitude(right)
 
assert magnitude([9,1]) == 29
assert magnitude([[1,2],[[3,4],5]]) == 143
assert magnitude([[[[0,7],4],[[7,8],[6,0]]],[8,1]]) == 1384
assert magnitude([[[[1,1],[2,2]],[3,3]],[4,4]]) == 445
assert magnitude([[[[3,0],[5,3]],[4,4]],[5,5]]) == 791
assert magnitude([[[[5,0],[7,4]],[5,5]],[6,6]]) == 1137
assert magnitude([[[[8,7],[7,7]],[[8,6],[7,7]]],[[[0,7],[6,6]],[8,7]]]) == 3488

Compute magnitude of sum of example.

In [10]:
magnitude(sum(parse(example)))

4140

Compute magnitude of sum of input.

In [11]:
magnitude(sum(parse(open('day-18-input.txt').read())))

4176

# Part two

In [12]:
import itertools

# Find the maximum magnitude of the addition of all length-2 permutations in input.
max([
    magnitude(add(*permutation)) for permutation in 
    itertools.permutations(parse(open('day-18-input.txt').read()), r=2)
])

4633